In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shapely
import getpass
import pymysql
import matplotlib.pyplot as plt
import geoplot as gplt
import folium
#import pandas.io.sql as psql


p = getpass.getpass()
connection = pymysql.connect(host = 'localhost', port = 3306, user = 'root', passwd = p) #, db='mysql')
#connection.autocommit(True)
cursor = connection.cursor()

In [2]:
def geometrify(wkt):
    if wkt is None:
        return np.nan
    else:
        return shapely.wkt.loads(wkt)

cursor.execute('use geo_analysis;')
brazil = pd.read_sql("SELECT id, UF, name, ST_AsText(geometry) AS geometry FROM cities;", con = connection)

brazil['geometry'] = brazil['geometry'].map(geometrify)
brazil = gpd.GeoDataFrame(brazil)
brazil

,id,UF,name,geometry
0,1100015,RO,Alta Floresta D'Oeste,"POLYGON ((-62.18209 -11.86686, -62.16230 -11.8..."
1,1100023,RO,Ariquemes,"POLYGON ((-62.53595 -9.73182, -62.50782 -9.754..."
2,1100031,RO,Cabixi,"POLYGON ((-60.39940 -13.45584, -60.40195 -13.4..."
3,1100049,RO,Cacoal,"POLYGON ((-61.00051 -11.39796, -61.01794 -11.4..."
4,1100056,RO,Cerejeiras,"POLYGON ((-61.50047 -13.00392, -61.47901 -13.0..."
...,...,...,...,...
5559,5222005,GO,Vianópolis,"POLYGON ((-48.43125 -16.62755, -48.42527 -16.6..."
5560,5222054,GO,Vicentinópolis,"POLYGON ((-49.85005 -17.57682, -49.84311 -17.5..."
5561,5222203,GO,Vila Boa,"POLYGON ((-47.11019 -14.67150, -47.11607 -14.6..."
5562,5222302,GO,Vila Propício,"POLYGON ((-48.75124 -14.90461, -48.75196 -14.9..."


In [3]:
ubs = pd.read_sql("SELECT cnes, UF, name, ST_AsText(geometry) AS geometry FROM ubs;", con = connection)

ubs['geometry'] = ubs['geometry'].map(geometrify)
ubs = gpd.GeoDataFrame(ubs)
ubs

,cnes,UF,name,geometry
0,108,PE,USF ALTO DOS INDIOS,POINT (-35.03210 -8.28389)
1,116,PE,USF CHARNECA II,POINT (-35.02819 -8.28353)
2,124,PE,USF SAO FRANCISCO I,POINT (-35.03500 -8.28700)
3,132,PE,USF ROSARIO,POINT (-35.03210 -8.28389)
4,140,PE,USF JUSSARAL,POINT (-35.03500 -8.28700)
...,...,...,...,...
39645,9999078,MG,PRBE PRESIDIO DE BOA ESPERANCA,POINT (-45.55373 -21.09526)
39646,9999280,PB,EMAD II SAO JOAO DO RIO DO PEIXE,POINT (-38.44900 -6.72900)
39647,9999396,BA,UNIDADE BASICA DE SAUDE IRIS LOPES,POINT (-39.59576 -16.31248)
39648,9999493,PR,UNIDADE DE SAUDE SANTAREM,POINT (-49.29147 -25.67153)


In [4]:
bolsa = pd.read_sql("SELECT * FROM bolsa_familia;", con = connection)
bolsa

,id,ibge,UF,y_m,qtd_ben_bas,qtd_ben_var,qtd_ben_bvj,qtd_ben_bvn,qtd_ben_bvg,qtd_ben_bsp
0,1,110001,RO,2020-01,1173,2416,220.0,43.0,57.0,234.0
1,2,110001,RO,2020-02,1153,2365,234.0,35.0,47.0,239.0
2,3,110001,RO,2020-03,1143,2378,241.0,37.0,48.0,230.0
3,4,110001,RO,2020-04,1158,2396,252.0,39.0,45.0,231.0
4,5,110001,RO,2020-05,1163,2375,264.0,36.0,55.0,234.0
...,...,...,...,...,...,...,...,...,...,...
66835,66836,530010,DF,2020-08,75819,123165,14245.0,683.0,1527.0,35417.0
66836,66837,530010,DF,2020-09,75865,120746,16621.0,455.0,2333.0,35375.0
66837,66838,530010,DF,2020-10,75939,120750,16620.0,81.0,2049.0,35406.0
66838,66839,530010,DF,2020-11,76025,120760,16620.0,62.0,1820.0,35434.0


In [5]:
cursor.close()
connection.close()

In [16]:
ubs['UF'].value_counts()

MG    5215
SP    4992
BA    3797
PE    2398
CE    2067
PR    1997
RS    1967
MA    1933
RJ    1732
SC    1657
PA    1492
PB    1442
GO    1350
PI    1055
RN     952
MT     914
AL     867
ES     802
MS     605
AM     579
SE     464
TO     429
RO     270
AC     241
DF     189
RR     122
AP     122
Name: UF, dtype: int64

In [ ]:
# Requer geopandas >= 1.10
brazil.explore(column = 'name', tooltip = 'name', popup = True,
               legend = False, style_kwds=dict(color="black"), tiles="CartoDB positron")

In [ ]:
#df.plot('drought_num', figsize = (12,12), cmap = 'YlOrRd', legend = True)

base = brazil.plot(figsize = (10,10), color = 'none', edgecolor = 'gainsboro', zorder = 1)
base.set_axis_off()
ubs.plot(ax = base, legend = False, zorder = 2, markersize = 2)#cmap = 'autumn_r', zorder = 2)